# Lab 1: Azure AI Foundry 리소스 배포하기 (Deploy Azure Resources)

## 개요 (Overview)

이 노트북에서는 Azure AI Foundry 기반 멀티 에이전트 시스템을 구축하기 위한 Azure 리소스를 배포합니다.

### 배포할 리소스 (Resources to Deploy)

- Azure AI Foundry Project
- Azure AI Search (RAG 지식 베이스용)
- Azure Container Apps Environment (에이전트 호스팅용)
- Azure Container Registry (이미지 저장용)
- Azure Key Vault (비밀 관리용)
- Azure Storage Account (데이터 저장용)
- Azure OpenAI Service (GPT-4 모델용)

### 아키텍처 (Architecture)

```
┌────────────────────────────────────────────────────────────┐
│                 Multi-Agent System                         │
│                                                            │
│  ┌─────────────────────────────────────────────┐          │
│  │          Main Agent                         │          │
│  │  (Task Analysis & Routing)                  │          │
│  └────────────┬────────────────┬────────────────┘          │
│               │                │                           │
│       ┌───────▼──────┐  ┌──────▼──────────┐               │
│       │  Tool Agent  │  │  Research       │               │
│       │  (MCP)       │  │  Agent (RAG)    │               │
│       └──────┬───────┘  └────────┬────────┘               │
│              │                   │                         │
│       ┌──────▼───────┐    ┌──────▼─────────┐              │
│       │  MCP Server  │    │  Azure AI      │              │
│       │  (Weather)   │    │  Search (RAG)  │              │
│       └──────────────┘    └────────────────┘              │
└────────────────────────────────────────────────────────────┘
```

### 학습 목표 (Learning Objectives)

1. ✅ Azure Developer CLI (azd)를 사용한 인프라 배포
2. ✅ Bicep 템플릿 기반 Infrastructure as Code 이해
3. ✅ Azure AI Foundry 리소스 구조 파악
4. ✅ 배포된 리소스 확인 및 연결 정보 수집

## 1. 사전 요구 사항 확인 (Prerequisites Check)

다음 도구들이 설치되어 있는지 확인합니다:

- Python 3.9 이상
- Azure CLI
- Azure Developer CLI (azd)
- Docker (Container Apps 배포 시 필요)

In [ ]:
import sys, subprocess, os
import platform

# 운영체제에 따라 PATH 설정 (macOS, Linux, Codespaces 모두 지원)
system = platform.system()
if system == 'Darwin':  # macOS
    # Homebrew 경로 추가 (Intel & Apple Silicon)
    extra_paths = '/opt/homebrew/bin:/usr/local/bin'
elif system == 'Linux':  # Linux / Codespaces
    # 일반적인 Linux 바이너리 경로
    extra_paths = '/usr/local/bin:/usr/bin:/home/codespace/.local/bin'
else:  # Windows
    extra_paths = ''

if extra_paths:
    os.environ['PATH'] = extra_paths + ':' + os.environ.get('PATH', '')

def check(cmd, name):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, timeout=3, env=os.environ)
        print(f"{'✓' if result.returncode == 0 else '✗'} {name}")
    except Exception as e:
        print(f"✗ {name}")

print("=== Prerequisites Check ===")
print(f"✓ Python {sys.version.split()[0]} ({system})")
check("az --version", "Azure CLI")
check("azd version", "Azure Developer CLI")
check("docker --version", "Docker")
print("="*50)

## 2. Azure 인증 (Azure Authentication)

Azure에 로그인하고 구독을 설정합니다. 브라우저가 열리고 인증을 진행합니다.

### 테넌트 ID 설정 안내

**대부분의 경우**: 테넌트 ID를 지정하지 않아도 됩니다. `tenant_id` 변수를 `"<YOUR_TENANT_ID>"` 또는 `None`으로 두고 실행하세요.

**테넌트 ID가 필요한 경우**:
- ✅ 여러 조직(회사)의 Azure 테넌트에 접근 권한이 있는 경우
- ✅ 특정 조직의 리소스로만 작업해야 하는 경우
- ✅ 로그인 시 "multiple tenants" 관련 오류가 발생하는 경우

**테넌트 ID 확인 방법**:
- Azure Portal → Azure Active Directory → 개요 → 테넌트 ID 복사
- 또는 조직 관리자에게 문의

In [ ]:
import subprocess, json

print("=== Azure Authentication ===\n🔐 Browser will open for authentication...\n")

# 테넌트 ID를 여기에 입력하세요 (선택사항)
# 예: tenant_id = "16b3c013-d300-468d-ac64-7eda0820b6d3"
tenant_id = "<YOUR_TENANT_ID>"  # 또는 None으로 설정하면 기본 테넌트 사용 

# 테넌트 ID가 설정되어 있으면 해당 테넌트로 로그인
if tenant_id and tenant_id != "<YOUR_TENANT_ID>":
    az = subprocess.run(f"az login --tenant {tenant_id}", shell=True, capture_output=True, text=True)
else:
    az = subprocess.run("az login", shell=True, capture_output=True, text=True)

print(f"{'✅' if az.returncode == 0 else '❌'} Azure CLI")

azd = subprocess.run("azd auth login", shell=True, capture_output=True, text=True)
print(f"{'✅' if azd.returncode == 0 else '❌'} Azure Developer CLI")

print("="*50)

In [ ]:
import subprocess, json

print("=== Verifying Azure Authentication ===\n")

result = subprocess.run("az account show", shell=True, capture_output=True, text=True, timeout=5)

if result.returncode == 0:
    account = json.loads(result.stdout)
    print(f"✅ Authenticated as: {account['user']['name']}")
    print(f"   Subscription: {account['name']}")
else:
    print("❌ Not authenticated - Run previous cell")

print("="*50)

## 3. 환경 변수 설정 (Environment Configuration)

Azure Developer CLI를 위한 환경을 설정합니다.

> **🌍 중요: 리전 변경도 여기 딱 1곳만!**  
> 아래 셀의 `location` 변수만 변경하면 됩니다.  
> Quota 부족이나 리전 제약이 있을 때만 수정하세요.  
> 
> **권장 리전:** `eastus` (기본값) → Quota 부족 시 `eastus2`, `westus`, `swedencentral` 사용

In [ ]:
# 프로젝트 루트 디렉토리로 이동 (절대 경로 사용)
import os
from pathlib import Path

# 프로젝트 루트 디렉토리 (노트북이 루트에 위치)
project_root = Path.cwd()

os.chdir(project_root)
print(f"✓ Project root: {os.getcwd()}")

# 디렉토리 구조 확인
checks = [
    ('infra/main.bicep', 'Bicep templates'),
    ('azure.yaml', 'azure.yaml'),
    ('src/foundry_agent', 'Agent source code')
]

for path, name in checks:
    if os.path.exists(path):
        print(f"✓ {name} found")
    else:
        print(f"✗ {name} NOT found")
        
print("="*50)

In [ ]:
import random, string, json

# 현재 Azure 구독 정보 가져오기
subscription_result = subprocess.run("az account show", shell=True, capture_output=True, text=True)

if subscription_result.returncode != 0:
    print("❌ Not authenticated! Please run Cell 5 (Azure Authentication) first.")
    raise Exception("Azure authentication required")

account_info = json.loads(subscription_result.stdout)
subscription_id = account_info['id']
subscription_name = account_info['name']

print(f"📋 Using Subscription: {subscription_name}")
print(f"   ID: {subscription_id}\n")

# =============================================================================
# 🌍 리전 설정 - Quota 부족 시 여기 1곳만 변경하면 됩니다!
# =============================================================================

location = "eastus"  # 👈 Quota 부족 시만 변경! (예: 'eastus2', 'westus', 'swedencentral')

# 환경 이름 생성 (자동)
random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))
environment_name = f"aiagent-{random_suffix}"

print(f"🌍 Region Configuration")
print("=" * 50)
print(f"Environment Name: {environment_name}")
print(f"Location:         {location}")
print("=" * 50)
print("\n💡 Tip: Quota 부족 오류 발생 시:")
print("   1. 위의 location 변수를 'eastus2', 'westus', 'swedencentral' 중 하나로 변경")
print("   2. 이 셀부터 다시 실행\n")

# azd 환경 생성 및 구독 설정
subprocess.run(f"azd env new {environment_name} --subscription {subscription_id} --no-prompt", shell=True, capture_output=True)
subprocess.run(f"azd env set AZURE_SUBSCRIPTION_ID {subscription_id}", shell=True, capture_output=True)
subprocess.run(f"azd env set AZURE_LOCATION {location}", shell=True, capture_output=True)
subprocess.run(f"azd env set AZURE_ENV_NAME {environment_name}", shell=True, capture_output=True)

print("✅ Environment configured successfully!")

In [ ]:
principal_result = subprocess.run("az ad signed-in-user show --query id -o tsv", shell=True, capture_output=True, text=True)

if principal_result.returncode == 0:
    principal_id = principal_result.stdout.strip()
    print(f"Principal ID: {principal_id}\n")
    
    subprocess.run(f"azd env set AZURE_PRINCIPAL_ID {principal_id}", shell=True, capture_output=True)
    subprocess.run(f"azd env set principalId {principal_id}", shell=True, capture_output=True)
    subprocess.run(f"azd env set environmentName {environment_name}", shell=True, capture_output=True)
    subprocess.run(f"azd env set location {location}", shell=True, capture_output=True)
    
    print("✓ All parameters configured")
else:
    print("❌ Failed to get Principal ID")

## 4. 모델 설정 (Configure Model)

배포할 Azure OpenAI 모델을 설정합니다. 기본값은 `gpt-5`이지만, 필요에 따라 다른 모델로 변경할 수 있습니다.

> **🎯 중요: 모델 변경은 여기 딱 1곳만!**  
> 아래 `model_name`과 `model_version` 변수만 변경하면 전체 프로젝트에 자동으로 반영됩니다.  
> 다른 파일은 수정할 필요가 없습니다!

### 지원 모델

| 모델명 | 버전 | 특징 |
|--------|------|------|
| `gpt-5` | `2025-08-07` | 논리 중심 및 다단계 작업 최적화 (기본값) |
| `gpt-5-chat` | `2025-08-07` | 고급 대화형, 멀티모달, 컨텍스트 인식 |
| `gpt-5-mini` | `2025-08-07` | 경량 버전, 비용 효율적 |
| `gpt-5-nano` | `2025-08-07` | 속도 최적화, 저지연 애플리케이션 |

**GPT-5 주요 기능:**
- Context Length: 200,000 토큰
- 멀티모달 입력 (텍스트, 이미지)
- 실시간 스트리밍 및 도구 지원
- 향상된 추론 능력 및 안전성

> **💡 참고:** 
> - GPT-5 패밀리는 모두 동일한 버전(`2025-08-07`)을 사용합니다.
> - 다른 모델 사용 시 해당 모델의 버전으로 변경하세요.
> - 버전 확인: [Azure OpenAI Models](https://learn.microsoft.com/azure/ai-services/openai/concepts/models)

In [ ]:
# =============================================================================
# 🎯 모델 설정 - 여기만 변경하면 전체 프로젝트에 자동 반영됩니다!
# =============================================================================

model_name = "gpt-4o"             # 👈 모델명 변경 (예: 'gpt-5-chat', 'gpt-5-mini', 'gpt-5-nano')
model_version = "2024-11-20"     # 👈 모델 버전 변경 (GPT-5 패밀리는 모두 동일)
model_capacity = 50              # TPM (Tokens Per Minute) 용량 (필요시 변경)

print("🤖 OpenAI Model Configuration")
print("=" * 50)
print(f"Model Name:     {model_name}")
print(f"Model Version:  {model_version}")
print(f"Capacity (TPM): {model_capacity}")
print("=" * 50)
print("\n✅ 이 설정이 전체 프로젝트에 적용됩니다!")
print("   → Azure OpenAI에 모델 배포")
print("   → Lab 3, 4의 .env 파일 자동 생성")
print("   → 모든 Agent가 동일 모델 사용\n")
print("💡 참고:")
print("   • GPT-5 패밀리: gpt-5, gpt-5-chat, gpt-5-mini, gpt-5-nano")
print("   • 모든 GPT-5 모델은 버전 2025-08-07 사용")
print("   • 다른 모델군 사용 시 해당 버전으로 변경 필요\n")

# azd 환경 변수로 설정
subprocess.run(f"azd env set openAiModelName {model_name}", shell=True, capture_output=True)
subprocess.run(f"azd env set openAiModelVersion {model_version}", shell=True, capture_output=True)
subprocess.run(f"azd env set openAiModelCapacity {model_capacity}", shell=True, capture_output=True)

## 5. 인프라 배포 (Deploy Infrastructure)

**⚠️ 중요 알림:**
- Azure OpenAI, AI Foundry 등의 리소스가 생성됩니다
- 예상 비용: 시간당 약 $5-10 (사용량에 따라 다름)
- **이 단계에서는 인프라만 생성하고, Container Apps는 생성하지 않습니다**

### 배포되는 리소스

1. **Resource Group** - 모든 리소스를 포함하는 그룹
2. **AI Foundry Project** - 에이전트 프로젝트 워크스페이스
3. **Azure OpenAI** - GPT-5 모델 (gpt-5, gpt-5-chat, gpt-5-mini, gpt-5-nano, text-embedding-3-large)
4. **Azure AI Search** - RAG를 위한 지식 베이스
5. **Container Apps Environment** - 에이전트 호스팅 환경 (빈 환경만 생성)
6. **Container Registry** - 도커 이미지 저장소
7. **Key Vault** - 비밀 및 키 관리
8. **Storage Account** - 데이터 저장소
9. **Log Analytics Workspace** - 로그 및 모니터링

### 배포하지 않는 리소스

- **Container Apps (MCP Server)** - Notebook 03에서 이미지와 함께 생성

**📝 노트:**
- Container Apps는 **Notebook 03**에서 `azd deploy` 실행 시 Docker 이미지와 함께 생성됩니다
- 지금은 이미지 pull 없이 빠르게 인프라만 구성합니다 (약 3-5분 소요)

In [ ]:
print("🚀 Starting Azure infrastructure deployment...")
print(f"Environment: {environment_name}")
print(f"Subscription: {subscription_id}")
print("⏱️  This will take approximately 3-5 minutes (no Container Apps)\n" + "="*50)

result = subprocess.run(
    f"azd provision --no-prompt --environment {environment_name}",
    shell=True,
    text=True,
    env={**os.environ, 'AZURE_SUBSCRIPTION_ID': subscription_id}
)

print("\n" + "="*50)
if result.returncode == 0:
    print("✅ Infrastructure deployment completed successfully!")
    print("\n📋 Created Resources:")
    print("   • AI Foundry Project & OpenAI Models")
    print("   • AI Search Service")
    print("   • Container Apps Environment (empty)")
    print("   • Container Registry")
    print("   • Key Vault, Storage, Monitoring")
    print("\n🚫 NOT Created (will be in Notebook 03):")
    print("   • Container Apps (MCP Server) - will be deployed with images")
    print("\n💡 Next Steps:")
    print("   1. Continue to Section 6: Verify deployed resources")
    print("   2. Notebook 02: Setup AI Search RAG")
    print("   3. Notebook 03: Build images & deploy Container Apps")
else:
    print(f"❌ Deployment failed with return code {result.returncode}")
    print("Check the error messages above for details.")
    print("\n⚠️  If you see 'InsufficientQuota' error:")    
    print("   📌 Alternative regions: eastus2, westus, swedencentral, northeurope")
    print("   1. Go back to Section 3 (Environment Configuration)")    
    print("   2. Change: location = 'eastus2'  # or 'westus', 'swedencentral'")
    print("   3. Re-run Section 3 and Section 5")

## 6. 배포 결과 확인 (Verify Deployment)

배포된 리소스와 연결 정보를 확인합니다.

In [ ]:
# azd 환경 변수 가져오기
env_result = subprocess.run(
    "azd env get-values",
    shell=True,
    capture_output=True,
    text=True
)

if env_result.returncode == 0:
    print("=== Deployed Resources Configuration ===\n")
    env_vars = {}
    for line in env_result.stdout.strip().split('\n'):
        if '=' in line:
            key, value = line.split('=', 1)
            # 따옴표 제거
            value = value.strip('"')
            env_vars[key] = value
            print(f"{key}: {value}")
    
    print("\n" + "="*50)
    print("✅ Environment variables loaded successfully")
else:
    print("❌ Failed to load environment variables")
    env_vars = {}

In [ ]:
# 리소스 그룹의 리소스 목록 확인
if 'AZURE_RESOURCE_GROUP' in env_vars:
    rg_name = env_vars['AZURE_RESOURCE_GROUP']
    
    resources_result = subprocess.run(
        f'az resource list --resource-group {rg_name} --query "[].{{Name:name, Type:type, Location:location}}" --output table',
        shell=True,
        capture_output=True,
        text=True
    )
    
    if resources_result.returncode == 0:
        print("=== Deployed Resources ===\n")
        print(resources_result.stdout)
    else:
        print("❌ Failed to list resources")
        print(f"Error: {resources_result.stderr}")
else:
    print("⚠️  Resource group name not found in environment variables")
    print("💡 Please run the previous cell (Cell 17) first to load environment variables.")

## 7. 주요 리소스 연결 정보 저장 (Save Configuration)

다음 노트북에서 사용할 연결 정보를 저장합니다.

In [ ]:
# 주요 연결 정보를 추출하여 config 파일로 저장
# 최소화된 환경변수만 사용 - AI Agent 실행에 필수적인 정보만 포함
config = {
    "resource_group": env_vars.get("AZURE_RESOURCE_GROUP", ""),
    "location": env_vars.get("AZURE_LOCATION", ""),
    "project_connection_string": env_vars.get("AZURE_AI_PROJECT_CONNECTION_STRING", ""),
    "search_endpoint": env_vars.get("AZURE_SEARCH_ENDPOINT", ""),
    "search_service_name": env_vars.get("AZURE_SEARCH_SERVICE_NAME", ""),
    "container_registry_endpoint": env_vars.get("AZURE_CONTAINER_REGISTRY_ENDPOINT", ""),
    "container_apps_environment_id": env_vars.get("AZURE_CONTAINER_APPS_ENVIRONMENT_ID", ""),
    "model_deployment_name": env_vars.get("openAiModelName").strip('"'),
    "model_version": env_vars.get("openAiModelVersion").strip('"'),
    "model_capacity": int(env_vars.get("openAiModelCapacity", "50").strip('"') or "50"),
}

# 프로젝트 루트에 config.json 저장
config_path = Path("config.json")

with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print("=== Configuration Saved (Minimized) ===")
print(f"Location: {config_path.absolute()}")
print("\nEssential Information:")
print(f"  - Resource Group: {config['resource_group']}")
print(f"  - Location: {config['location']}")
print(f"  - AI Project Connection: {'✓ Set' if config['project_connection_string'] else '✗ Missing'}")
print(f"  - Search Endpoint: {config['search_endpoint']}")
print(f"  - Container Registry: {config['container_registry_endpoint']}")
print("\n✅ Minimized configuration saved successfully!")
print("\n💡 Note: Project connection string contains all AI Foundry info (project, OpenAI, etc.)")

## 8. Azure Portal에서 확인 (Verify in Azure Portal)

Azure Portal에서 배포된 리소스를 확인할 수 있습니다.

In [ ]:
# Azure Portal 링크 생성
resource_group = env_vars.get("AZURE_RESOURCE_GROUP", "")
subscription_id = env_vars.get("AZURE_SUBSCRIPTION_ID", "")

if resource_group and subscription_id:
    portal_url = f"https://portal.azure.com/#@/resource/subscriptions/{subscription_id}/resourceGroups/{resource_group}/overview"
    print("✅ Azure Portal에서 리소스 그룹 확인:")
    print(f"🔗 {portal_url}")
    print("\n🎯 주요 리소스:")
    print("  • AI Project: 에이전트 프로젝트")
    print("  • Azure OpenAI: GPT-4 모델")
    print("  • AI Search: RAG 지식 베이스")
    print("  • Container Apps Environment: 에이전트 실행 환경")
else:
    print("⚠️ 환경 변수에서 리소스 그룹 정보를 찾을 수 없습니다.")

## 9. 배포 완료 및 요약 (Deployment Summary)

축하합니다! Azure AI Foundry 인프라 배포를 성공적으로 완료했습니다.

### 배포된 리소스 (Deployed Resources)

| 리소스 타입 | 용도 | 상태 |
|------------|------|------|
| AI Project | 에이전트 프로젝트 워크스페이스 | ✅ 생성됨 |
| Azure OpenAI | GPT-5, Embeddings 모델 | ✅ 배포됨 |
| AI Search | RAG 지식 베이스 (벡터 검색) | ✅ 생성됨 |
| Container Registry | Docker 이미지 저장소 | ✅ 생성됨 |
| Container Apps Environment | 에이전트 호스팅 환경 | ✅ 생성됨 |
| Key Vault | 시크릿 및 연결 문자열 관리 | ✅ 생성됨 |
| Storage Account | 데이터 및 로그 저장 | ✅ 생성됨 |
| Log Analytics | 모니터링 및 로깅 | ✅ 생성됨 |
| Application Insights | 애플리케이션 성능 모니터링 | ✅ 생성됨 |

### 📝 중요 사항

**현재 완료된 작업:**
- ✅ Azure 인프라 리소스 생성 완료
- ✅ Container Apps Environment 준비 완료
- ✅ 역할 할당 및 권한 설정

**아직 완료되지 않은 작업:**
- ⏸️ Docker 이미지 빌드
- ⏸️ Container Registry에 이미지 푸시
- ⏸️ Container Apps 생성 및 배포

→ 이 작업들은 **Notebook 03**에서 진행됩니다.

### 다음 단계 (Next Steps)

이제 다음 노트북으로 순서대로 진행하세요:

1. **Notebook 2: RAG 지식 베이스 구성** (`02_setup_ai_search_rag.ipynb`)
   - Azure AI Search 인덱스 생성
   - 샘플 데이터 임베딩 및 업로드
   - 하이브리드 검색 (벡터 + 키워드) 테스트

2. **Notebook 3: 에이전트 이미지 빌드 및 배포** (`03_deploy_foundry_agent.ipynb`)
   - MCP Server Docker 이미지 빌드
   - Container Registry에 이미지 푸시
   - Container Apps 생성 및 배포
   - 멀티 에이전트 시스템 테스트